In [152]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

In [153]:
time = []

start_time = 0.00

end_time = 4.50 

lower = np.arange(start_time, 4.5, 0.05)

upper = np.arange(0.05, end_time+0.05, 0.05) # add 0.05 because np.arange excludes final value from the given range

for i in range(90):
    add = np.random.uniform(upper[i], lower[i])
    time.append(add)
    
dictionary = {
    "time" : time, 
    "speed" : np.random.random(90)
}

df = pd.DataFrame(dictionary)

display(df.head())

print(f"The number of taps is {len(df)}")

,time,speed
0,0.046717,0.266256
1,0.095686,0.763774
2,0.109400,0.355971
3,0.195380,0.679051
4,0.202975,0.146853


The number of taps is 90


In [154]:
light_duration = 1

off_duration = 0.1

light_on = np.arange(start_time+off_duration, end_time+off_duration, 1+off_duration)

light_off = np.arange(1.1, end_time+off_duration, 0.1+light_duration)

light_pulse_count = 4 

light_pulses = list(zip(light_on, light_off)) 

print("light pulses = {}".format(light_pulses))

light pulses = [(0.1, 1.1), (1.2000000000000002, 2.2), (2.3000000000000003, 3.3000000000000003), (3.4000000000000004, 4.4)]


In [183]:
def light_pulses_filter(df, light_pulses):
    """
    This function filters out the time points during which there is a light pulse and feeds them 
    to a new dataframe with a column of pulse numbers. 
    
    :param df: a dataframe containing a time column 
    :type: pandas.core.frame.DataFrame
    :param light_pulses: a list of tuples of time ranges corresponding to light pulses
    :type: list
    
    :return: dataframe of light pulses containing time and pulse columns
    :type: pandas.core.frame.DataFrame
    """
    # start pulse count 
    pulse_num = 1
    print("Processing pulse {} out of {}".format(pulse_num, len(light_pulses)))
    # initialize the dataframe with the first pulse
    pulses_df = df[df["time"].between(light_pulses[0][0], light_pulses[0][1])]
    # initialize pulse column with ones
    pulses_df["pulse"] = 1
    df["pulse"] = 1
    
    for pulse in light_pulses[1:]:
        pulse_num += 1
        print("Processing pulse {} out of {}".format(pulse_num, len(light_pulses)))
        light_on = pulse[0]
        light_off = pulse[1]
        # get slice of time column between light_on and light_off, i.e. the times during which there is a pulse
        df_slice = df[df["time"].between(light_on, light_off)]
        # add the pulse to the pulses dataframe
        pulses_df = pd.concat([pulses_df, df_slice])
        # get start and end indices of the light_on and light_off times 
        start_ind = pulses_df[pulses_df["time"] == df_slice["time"].iloc[0]].index.astype(int)[0]
        end_ind = pulses_df[pulses_df["time"] == df_slice["time"].iloc[-1]].index.astype(int)[0]
        # use the start and end indices to label the times with their corresponding pulse number
        pulses_df["pulse"].mask(pulses_df["time"].between(light_on, light_off), other=pulse_num, inplace=True)
        
    return pulses_df

In [184]:
new_df = light_pulses_filter(df, light_pulses)

display(new_df)

Processing pulse 1 out of 4
Processing pulse 2 out of 4
Processing pulse 3 out of 4
Processing pulse 4 out of 4


,time,speed,pulse
2,0.109400,0.355971,1
3,0.195380,0.679051,1
4,0.202975,0.146853,1
5,0.265445,0.812703,1
6,0.323800,0.234704,1
7,0.356942,0.605298,1
8,0.410821,0.447202,1
9,0.453213,0.134852,1
10,0.524041,0.181631,1
11,0.568959,0.336911,1
